## Global Setting

In [ ]:
import numpy as np
import torch

# True if you want to get dynamic output in notebook block, else False
SHOW_PLOT_IN_BLOCK: bool = False


# To keep the strategy the same, I use BGD uniformly
LEARNING_RATE: float = 1e-2
MAX_STEPS: int = 1500
EVAL_FREQ: int = 10
DNN_HIDDEN_UNITS: str = "20"

GLOBAL_SEED: int = 42

np.random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.cuda.manual_seed_all(GLOBAL_SEED)

## Task 2

### Show data


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import *


class CmpDataGenerator:
    def __init__(self, type_id: int, n_samples: int = 1000, random_seed: int = 42):
        self.type_id: int = type_id
        self.type_name: str = ""
        self.random_seed: int = random_seed
        self.n_samples: int = n_samples

        # input and label
        x: np.ndarray
        y: np.ndarray
        if type_id == 1:
            self.type_name = "moon"
            x, y = make_moons(
                n_samples=self.n_samples, random_state=self.random_seed, noise=0.05
            )
        elif type_id == 2:
            self.type_name = "blobs"
            x, y = make_blobs(
                n_samples=self.n_samples,
                random_state=self.random_seed,
                centers=2,
                cluster_std=3,
            )
        elif type_id == 3:
            self.type_name = "gaussian_quantiles"
            x, y = make_gaussian_quantiles(
                n_samples=self.n_samples, random_state=self.random_seed, n_classes=2
            )
        y = OneHotEncoder(sparse_output=False).fit_transform(y.reshape(-1, 1))
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(
            x, y, random_state=self.random_seed, train_size=0.8, test_size=0.2
        )

    def save_data_fig(self, show: bool = SHOW_PLOT_IN_BLOCK):
        train_pos = self.x_train[np.where(self.y_train[:, 0] == 1)]
        train_neg = self.x_train[np.where(self.y_train[:, 0] == 0)]
        test_pos = self.x_test[np.where(self.y_test[:, 0] == 1)]
        test_neg = self.x_test[np.where(self.y_test[:, 0] == 0)]

        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.scatter(
            train_pos[:, 0], train_pos[:, 1], c="blue", label="positive train point"
        )
        plt.scatter(
            train_neg[:, 0], train_neg[:, 1], c="red", label="negative train point "
        )
        plt.legend()
        title = self.type_name + " train points"
        plt.title(title)

        plt.subplot(1, 2, 2, sharex=plt.gca(), sharey=plt.gca())
        plt.scatter(
            test_pos[:, 0], test_pos[:, 1], c="blue", label="positive test point"
        )
        plt.scatter(
            test_neg[:, 0], test_neg[:, 1], c="red", label="negative test point "
        )
        plt.legend()
        title = self.type_name + " test points"
        plt.title(title)

        plt.tight_layout()

        path = "../Report/img/Part1/data_" + self.type_name + ".png"
        plt.savefig(path)
        if show:
            plt.show()
        plt.close()


dataset1: CmpDataGenerator = CmpDataGenerator(1)
dataset2: CmpDataGenerator = CmpDataGenerator(2)
dataset3: CmpDataGenerator = CmpDataGenerator(3)

dataset1.save_data_fig(show=SHOW_PLOT_IN_BLOCK)
dataset2.save_data_fig(show=SHOW_PLOT_IN_BLOCK)
dataset3.save_data_fig(show=SHOW_PLOT_IN_BLOCK)

### Compare Accuracy

#### Define Plot Function

In [ ]:
def save_fig_acc(
    steps_numpy: list[int],
    steps_torch: list[int],
    train_numpy: list[float],
    train_torch: list[float],
    test_numpy: list[float],
    test_torch: list[float],
    file_name: str,
    show: bool = SHOW_PLOT_IN_BLOCK,
) -> None:
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(steps_numpy, train_numpy, label="Numpy Train Acc")
    plt.plot(steps_torch, train_torch, label="Pytorch Train Acc")
    plt.title(f"{file_name} Training Accuracy")
    plt.xlabel("Step")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(steps_numpy, test_numpy, label="Numpy Test Acc")
    plt.plot(steps_torch, test_torch, label="Pytorch Test Acc")
    plt.title(f"{file_name} Testing Accuracy")
    plt.xlabel("Step")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    path = "../Report/img/Part1/acc_" + file_name + ".png"
    plt.savefig(path)
    if show:
        plt.show()
    plt.close()


def save_fig_loss(
    steps_numpy: list[int],
    steps_torch: list[int],
    train_numpy: list[float],
    train_torch: list[float],
    test_numpy: list[float],
    test_torch: list[float],
    file_name: str,
    show: bool = SHOW_PLOT_IN_BLOCK,
) -> None:
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(steps_numpy, train_numpy, label="Numpy Train Loss")
    plt.plot(steps_torch, train_torch, label="Pytorch Train Loss")
    plt.title(f"{file_name} Training Loss")
    plt.xlabel("Step")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(steps_numpy, test_numpy, label="Numpy Test Loss")
    plt.plot(steps_torch, test_torch, label="Pytorch Test Loss")
    plt.title(f"{file_name} Testing Loss")
    plt.xlabel("Step")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    path = "../Report/img/Part1/loss_" + file_name + ".png"
    plt.savefig(path)
    if show:
        plt.show()
    plt.close()

### Generate Result


In [ ]:
import numpy_train_mlp
import pytorch_train_mlp

for dataset in [dataset1, dataset2, dataset3]:
    data_in = dataset.x_train, dataset.y_train, dataset.x_test, dataset.y_test
    steps_numpy, train_acc_numpy, test_acc_numpy, train_loss_numpy, test_loss_numpy = (
        numpy_train_mlp.train(
            data_in,
            dnn_hidden_units=DNN_HIDDEN_UNITS,
            learning_rate=LEARNING_RATE,
            max_steps=MAX_STEPS,
            eval_freq=EVAL_FREQ,
        )
    )
    steps_torch, train_acc_torch, test_acc_torch, train_loss_torch, test_loss_torch = (
        pytorch_train_mlp.train(
            data_in,
            dnn_hidden_units=DNN_HIDDEN_UNITS,
            learning_rate=LEARNING_RATE,
            max_steps=MAX_STEPS,
            eval_freq=EVAL_FREQ,
        )
    )
    save_fig_acc(
        steps_numpy,
        steps_torch,
        train_acc_numpy,
        train_acc_torch,
        test_acc_numpy,
        test_acc_torch,
        dataset.type_name,
    )
    save_fig_loss(
        steps_numpy,
        steps_torch,
        train_loss_numpy,
        train_loss_torch,
        test_loss_numpy,
        test_loss_torch,
        dataset.type_name,
    )

print("Generate Done!")

Generate Done!


## Static Show Images

### Moon

Data shape

![data_moon](../Report/img/Part1/data_moon.png)

Accuracy Curve

![acc_moon](../Report/img/Part1/acc_moon.png)

Loss Curve

![loss_moon](../Report/img/Part1/loss_moon.png)

### Blobs

Data shape

![data_blobs](../Report/img/Part1/data_blobs.png)

Accuracy Curve

![acc_blobs](../Report/img/Part1/acc_blobs.png)

Loss Curve

![loss_blobs](../Report/img/Part1/loss_blobs.png)


### Gaussian Quantiles

Data shape

![data_gaussian_quantiles](../Report/img/Part1/data_gaussian_quantiles.png)

Accuracy Curve

![acc_gaussian_quantiles](../Report/img/Part1/acc_gaussian_quantiles.png)

Loss Curve

![loss_gaussian_quantiles](../Report/img/Part1/loss_gaussian_quantiles.png)


